In [172]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [173]:
import sys; sys.path.insert(0, '..') # add parent folder path where lib folder is

In [174]:
from ml.models.base.tabular_dataset import TabularDataset
from ml.preprocess import data

In [175]:
from torch.utils.data import random_split, DataLoader 

import torch
# from torch.nn import functional as F
from torchmetrics import Accuracy
from torch import nn
import pytorch_lightning as pl


In [176]:
from sklearn.model_selection import train_test_split

X, y = data.get_dataset(f_name = '20220319_covid_merge_processed.csv' , sep = ',')
# X_tilda = X.drop(brisk_xgboost.DROP_LIST, axis = 1)


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=123)


In [189]:
from torch.utils.data import DataLoader, Subset


TEST_SIZE = 0.1
BATCH_SIZE = 64
SEED = 42


# generate indices: instead of the actual data we pass in integers instead
train_indices, test_indices, _, _ = train_test_split(
    X, y,
    # stratify=data.targets,
    test_size=TEST_SIZE,
    random_state=SEED
)

# generate subset based on indices
train_split = Subset(data, train_indices)
test_split = Subset(data, test_indices)

# create batches
train_batches = DataLoader(X, batch_size=BATCH_SIZE, shuffle=True)
test_batches = DataLoader(test_split.dataset, batch_size=BATCH_SIZE)

In [191]:
train_batches = DataLoader(X, batch_size=32, shuffle=True)

In [193]:
train_batches.size()

AttributeError: 'DataLoader' object has no attribute 'size'

In [194]:

for batch_idx, (data, target) in enumerate(train_batches):
    print(batch_idx)

KeyError: 82

In [63]:
class DataModuleClass(pl.LightningDataModule):
    def __init__(self, X, y, batch_size):
        #Define required parameters here
        super().__init__()
        self.X = X
        self.y = y
        self.TEST_SIZE = 0.3
        self.VAL_SIZE = 0.2
        self.BATCH_SIZE = batch_size
        self.SEED = 42
        self.train_split = None
        self.test_split = None
        self.val_split = None

        
    def setup(self, stage=None):
        # Define steps that should be done on 
        # every GPU, like splitting data, applying
        # transform etc.
        # generate indices: instead of the actual data we pass in integers instead
        train_indices, test_indices, _, _ = train_test_split(
            self.X, 
            self.y,
            # stratify=data.targets,
            test_size=self.TEST_SIZE,
            random_state=self.SEED
        )

        # generate subset based on indices
        self.train_split = Subset(self.X, train_indices)
        self.test_split = Subset(self.X, test_indices)

        # print(self.test_split)
    
    def train_dataloader(self):
        # Return DataLoader for Training Data here
        return DataLoader(self.train_split, batch_size=self.BATCH_SIZE, shuffle=True)
        
        
    def test_dataloader(self):
        # Return DataLoader for Testing Data here
        return DataLoader(self.test_split, batch_size=self.BATCH_SIZE, shuffle=True)


In [64]:
# Create MNIST DataModule instance
data_module = DataModuleClass(X, y, batch_size=64)
data_module.setup()
train = data_module.train_dataloader()

TypeError: 'DataLoader' object is not subscriptable

In [60]:
for xb, yb in train:
    print(xb.shape)

KeyError: 48

In [49]:
image = iter(train)

In [43]:
train

In [195]:


class CDataset(Dataset):
    
    def __init__(self, x, y):
        self.x = x
        self.y = y
    
    def __getitem__(self, index):
        #sample = {
        #    'feature': torch.tensor([self.x[index]], dtype=torch.float32), 
        #    'label': torch.tensor([self.y[index]], dtype=torch.long)}
        #return sample    
        num_array = torch.tensor([self.x.loc[index]], dtype=torch.float32)
        # num_array = torch.tensor(num_array, dtype = torch.float32)
        # label_array = self.df[self.label_col].iloc[idx]
        label_array = torch.tensor([self.y.loc[index]], dtype=torch.float32)
        return num_array, label_array

    def __len__(self):
        return len(self.x)

In [203]:
cdataset = CDataset(X, y)
dl = DataLoader(cdataset, batch_size=1, shuffle=True)

In [204]:
# dl.size()
next(iter(dl))

KeyError: 122

In [127]:
class DataModuleClass(pl.LightningDataModule):
    def __init__(self, X, y, batch_size):
        #Define required parameters here
        super().__init__()

        self.X = X
        self.y = y
        self.TEST_SIZE = 0.3
        self.VAL_SIZE = 0.2
        self.BATCH_SIZE = batch_size
        self.SEED = 42
        self.train = None
        self.test = None
        self.val = None
        self.dataset = TabularDataset(X, y)

        
    def setup(self, stage=None):
        # Define steps that should be done on 
        # every GPU, like splitting data, applying
        # transform etc.
        # generate indices: instead of the actual data we pass in integers instead
        train_indices, test_indices, _, _ = train_test_split(
            self.X, 
            self.y,
            # stratify=data.targets,
            test_size=self.TEST_SIZE,
            random_state=self.SEED
        )

        # generate subset based on indices
        self.train_split = Subset(self.X, train_indices)
        self.test_split = Subset(self.X, test_indices)

        data_len = self.X.shape[0]

        train_size = int((1 - self.TEST_SIZE) * data_len)
        test_size = data_len - train_size
        
        self.train, self.test = random_split(self.dataset, [train_size, test_size])

        print(self.train.dataset)

    
    def train_dataloader(self):
        # Return DataLoader for Training Data here
        return DataLoader(self.X, batch_size=self.BATCH_SIZE, shuffle=True)
        # print(self.train.indices.shape)
        return self.train
        
        
    def test_dataloader(self):
        # Return DataLoader for Testing Data here
        # return DataLoader(self.test_split, batch_size=self.BATCH_SIZE, shuffle=True)
        return self.test


In [147]:
train_loader_manual = torch.utils.data.DataLoader(
    X,
    batch_size=1,
    num_workers=0,
    shuffle=True
)


training_data_size = 0.8
train_size = int(training_data_size * len(train_loader_manual))
test_size = len(train_loader_manual) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(X, [train_size, test_size])

In [166]:
train_loader_manual = torch.utils.data.DataLoader(
    train_dataset.dataset,
    batch_size=32,
    num_workers=0,
    shuffle=True
)

In [167]:
for batch_idx, (data, target) in enumerate(train_loader_manual):
    print(batch_idx)

KeyError: 37

In [168]:
data_module = DataModuleClass(X, y, batch_size=64)
data_module.setup()
train = data_module.train_dataloader()

In [170]:
for batch_idx, (data, target) in enumerate(train):
    print(batch_idx)

KeyError: 11

In [90]:
test.dataset.x

,location,new_cases_per_million,population_cov,life_expectancy_cov,Population,Area_km2,Density_km2,Year_x,Meningitis,Neoplasms,"Fire, heat, and hot substances",Malaria,Drowning,Interpersonal violence,HIV/AIDS,Drug use disorders,Tuberculosis,Road injuries,Maternal disorders,Lower respiratory infections,Neonatal disorders,Alcohol use disorders,Exposure to forces of nature,Diarrheal diseases,Environmental heat and cold exposure,Nutritional deficiencies,Self-harm,Conflict and terrorism,Diabetes mellitus,Poisonings,Protein-energy malnutrition,Cardiovascular diseases,Chronic kidney disease,Chronic respiratory diseases,Cirrhosis and other chronic liver diseases,Digestive diseases,Acute hepatitis,Alzheimer's disease and other dementias,Parkinson's disease,Total,air_polution
0,0,4171.944,39835428.0,64.83,37171921.0,252071.0,57.0,2019.0,0.004205,0.057159,0.001305,0.001426,0.004538,0.013491,0.000855,0.001092,0.009757,0.022205,0.010863,0.050299,0.063760,0.000395,0.000344,0.011622,0.000159,0.003347,0.004339,0.065358,0.012959,0.001412,0.003255,0.166779,0.015165,0.019052,0.010215,0.018640,0.005219,0.004775,0.001507,0.585498,52.99812
1,1,92109.327,2872934.0,78.57,2882740.0,11100.0,100.0,2019.0,0.000451,0.163213,0.000624,0.000000,0.001249,0.001977,0.000069,0.001006,0.000382,0.008429,0.000104,0.015853,0.005585,0.000624,0.001769,0.000243,0.000139,0.000173,0.005273,0.000000,0.006071,0.000382,0.000104,0.447630,0.011413,0.028272,0.011552,0.018281,0.000035,0.031810,0.008603,0.771315,18.45981
2,2,5782.052,44616626.0,76.88,42228408.0,919590.0,18.0,2019.0,0.000691,0.056398,0.001852,0.000000,0.001246,0.001087,0.000625,0.001246,0.001054,0.026170,0.001511,0.013702,0.020735,0.000263,0.000000,0.001248,0.000095,0.000270,0.003588,0.000031,0.012617,0.000831,0.000218,0.231908,0.019421,0.017827,0.009579,0.015151,0.000613,0.012335,0.003038,0.455347,32.91557
3,3,473136.492,77354.0,83.73,77006.0,181.0,165.0,2019.0,0.000000,0.298678,0.000000,0.000000,0.000000,0.000000,0.003896,0.000000,0.000000,0.010389,0.000000,0.025972,0.000000,0.001299,0.000000,0.001299,0.000000,0.000000,0.010389,0.000000,0.011687,0.000000,0.000000,0.219463,0.020778,0.050645,0.015583,0.035062,0.001299,0.046750,0.009090,0.762278,9.51265
4,4,2898.709,33933611.0,61.15,30809787.0,481351.0,25.0,2019.0,0.008179,0.041516,0.001665,0.035002,0.002574,0.003161,0.054535,0.000260,0.038144,0.030033,0.006715,0.041490,0.059036,0.000685,0.000097,0.041987,0.000370,0.009786,0.006258,0.000052,0.013090,0.001405,0.009432,0.083493,0.007997,0.012769,0.018390,0.029117,0.000798,0.003710,0.000867,0.562613,28.02152
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
186,186,17289.124,28704947.0,72.06,28887118.0,352143.0,32.0,2019.0,0.000879,0.120369,0.000679,0.002309,0.001814,0.037885,0.006404,0.000312,0.002859,0.021487,0.001720,0.019230,0.011555,0.000651,0.000000,0.004909,0.000038,0.001911,0.007997,0.000000,0.037370,0.000284,0.001824,0.212932,0.036020,0.024447,0.016243,0.029775,0.000073,0.021854,0.003874,0.627702,22.21524
187,187,23856.549,98168829.0,75.40,95545962.0,127782.0,289.0,2019.0,0.000782,0.117811,0.000636,0.000062,0.006382,0.001326,0.006697,0.001231,0.019552,0.025279,0.000245,0.022340,0.006770,0.001047,0.000005,0.001750,0.000086,0.000998,0.008492,0.000000,0.030761,0.000979,0.000099,0.251357,0.022487,0.037918,0.024739,0.032765,0.000705,0.020367,0.004458,0.648126,20.19471
188,188,366.236,30490639.0,66.12,28498683.0,203848.0,54.0,2019.0,0.001681,0.044497,0.002944,0.017857,0.002958,0.002877,0.001218,0.000916,0.003611,0.041114,0.005983,0.022068,0.066607,0.000288,0.000000,0.023208,0.000151,0.003544,0.005474,0.047027,0.006369,0.001579,0.003302,0.195630,0.009390,0.020134,0.009776,0.017113,0.001835,0.006074,0.001449,0.566672,44.64506
189,189,16236.562,18920657.0,63.89,17351708.0,290583.0,23.0,2019.0,0.011901,0.056415,0.001879,0.027530,0.002069,0.009313,0.129901,0.000346,0.040665,0.013169,0.004634,0.047200,0.

In [68]:
dataset = MyDataset(X, y)

In [70]:
X.shape

(191, 41)

In [71]:
from torch.utils.data import random_split
train, test = random_split(dataset, [100, 91])

In [97]:
class BriskModel(pl.LightningModule): 
    def __init__(self, hidden_size = 40): 
        super(BriskModel, self).__init__() 

        # Defining learning rate
        self.lr = 0.089

        # Defining our model architecture
        self.layer_0 = nn.BatchNorm1d(40)
        self.layer_1 = nn.Linear(hidden_size, 243)
        self.layer_2 = nn.Linear(hidden_size, 90)
        self.layer_3 = nn.Linear(hidden_size, 22)
        self.layer_final = nn.Linear(hidden_size, 1)
        self.relu = nn.ReLU()

        # nn.init.xavier_normal_(self.layer_1.weight)
        # self.layer_2 = nn.BatchNorm1d(16)
        # self.layer_3 = nn.Linear(16, 96)
        # nn.init.xavier_normal_(self.layer_3.weight)
        # self.layer_4 = nn.Linear(96, 1)
        # nn.init.xavier_normal_(self.layer_4.weight)
        # self.out = nn.Flatten(0, 1)
                  
          
        # Defining loss 
        ##self.loss = nn.BCEWithLogitsLoss()
        # self.loss = nn.MSELoss()
    
        self.val_accuracy = Accuracy()
        self.test_accuracy = Accuracy()


        
    def forward(self, x):
#        return self.model(x)


        # Defining the forward pass of the model
        # print(x)
        # batch_size, _= x.size() 
        # x = x.view(batch_size, -1) 
        
        # Pass the tensor through the layers
        #x = self.layer_1(x)
        #x = nn.relu(x)
        #x = self.layer_2(x)
        #x = self.layer_3(x)
        #x = F.relu(x)
        #x = self.layer_4(x)
        #x = F.sigmoid(x)
        #x = self.out(x)
        #return x

        x = self.layer_0(x)
        x = self.relu(self.layer_1(x))
        x = self.relu(self.layer_2(x))
        x = self.relu(self.layer_3(x))
        x = self.layer_out(self.layer_final(x))
        return x
    
    def configure_optimizers(self):
          # Defining and returning the optimizer for our model
        # with the defines parameters
        #return torch.optim.SGD(self.parameters(), lr = self.lr) 
        return torch.optim.Adam(self.parameters(), lr=self.lr)
    
    def training_step(self, train_batch, batch_idx): 
        # Defining training steps for our mode

        x, y = train_batch
        pred = self.forward(x)
        loss = nn.MSELoss(pred, y)
        return loss

        x, y = train_batch
        logits = self.forward(x)#.to(torch.float32)
        
        logits = torch.round(torch.sigmoid(logits))
        
        # logits = logits.to(torch.float32)
        y = y.to(torch.float32)
        # print('train_y:' + str(len(y)) + ' train_prec_y:' + str(len(logits)))
        # print('train:' + str(max(logits)) + ' min:' + str(min(logits)))
        loss = self.loss(logits, y)
        
        bin_acc = binary_acc(logits, y)  
        
        print('train los:' + str(loss) + ' train acc: '+ str(bin_acc))
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        preds = self(x)
        loss = nn.MSELoss(preds, y)
        # preds = torch.argmax(logits, dim=1)
        self.val_accuracy.update(preds, y)

        # Calling self.log will surface up scalars for you in TensorBoard
        self.log("val_loss", loss, prog_bar=True)
        self.log("val_acc", self.val_accuracy, prog_bar=True)



    def test_step(self, batch, batch_idx):
        x, y = batch
        preds = self(x)
        loss = nn.MSELoss(preds, y)
        # preds = torch.argmax(logits, dim=1)
        self.test_accuracy.update(preds, y)

        # Calling self.log will surface up scalars for you in TensorBoard
        self.log("test_loss", loss, prog_bar=True)
        self.log("test_acc", self.test_accuracy, prog_bar=True)


In [98]:
clf = BriskModel() 
# data = DataModuleClass() 
data = DataModuleClass(X, y, batch_size=64)
trainer = pl.Trainer(min_epochs=10, max_epochs=100)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [99]:
data

In [100]:
trainer.fit(clf, data)

Missing logger folder: /mnt/c/Users/rwmas/GitHub/xai/xai_api/app/notebooks/lightning_logs

  | Name          | Type        | Params
----------------------------------------------
0 | layer_0       | BatchNorm1d | 80    
1 | layer_1       | Linear      | 10.0 K
2 | layer_2       | Linear      | 3.7 K 
3 | layer_3       | Linear      | 902   
4 | layer_final   | Linear      | 41    
5 | relu          | ReLU        | 0     
6 | val_accuracy  | Accuracy    | 0     
7 | test_accuracy | Accuracy    | 0     
----------------------------------------------
14.7 K    Trainable params
0         Non-trainable params
14.7 K    Total params
0.059     Total estimated model params size (MB)


TypeError: Expected data to be int, Sequence or Mapping, but got Subset